In [1]:
import pandas as pd
import numpy as np
import os
from config import C_COMMON, C_SINGLE, C_GRID
from copy import deepcopy
import time
import matplotlib.pyplot as plt

# import models
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('/home/yusukemh/sadow_lts/personal/yusukemh/pi_casc/processed_datasets/dataset_6grid.csv')
# split
df_train = df.query('year < 1984')
df_valid = df.query('1984 <= year < 1997')
df_test = df.query('1997 <= year')

assert df_train.shape[0] + df_valid.shape[0] + df_test.shape[0] == df.shape[0]

# Define columns: lat, lon and 16 reanalysis variables
columns = deepcopy(C_SINGLE + C_COMMON)
for r in ['fold', 'skn', 'year', 'month', 'data_in', 'season_wet', 'elevation']:
    columns.remove(r)

In [3]:
Xtrain, Ytrain = np.array(df_train[columns]), np.array(df_train['data_in'])
Xvalid, Yvalid = np.array(df_valid[columns]), np.array(df_valid['data_in'])
Xtest, Ytest = np.array(df_test[columns]), np.array(df_test['data_in'])

In [32]:
xgboost = XGBRegressor(
        n_estimators=300,
        learning_rate=0.09153907009972553,
        max_depth=8,
        n_jobs=-1,
        # subsample=0.9,
        seed=np.random.randint(100),
        verbosity=0,
        gamma=1.5
    )

xgboost.fit(Xtrain, Ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=1.5, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.09153907009972553,
             max_delta_step=0, max_depth=8, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=300, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=9, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=9, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [33]:
mean_squared_error(Ytest, xgboost.predict(Xtest), squared=True)

16.091513139630926

In [18]:
# base
mean_squared_error(Ytest, xgboost.predict(Xtest))

16.157175348988595